In [8]:
import cirq
import numpy as np
from sklearn.preprocessing import normalize

In [25]:
def beta(s, j, x):
    index_num = (2*j-1)*(2**(s-1))
    index_den = (j-1)*(2**s)
    
    num = np.sqrt(np.sum(abs(x[index_num : index_num+2**(s-1)])**2))
    den = np.sqrt(np.sum(abs(x[index_den : index_den+2**(s)])**2))
    
    if den == 0:
        beta = 0
    else:
        beta = 2*np.arcsin(num/den)
    return beta

In [26]:
def locate_x(curr_j, prev_j, length):
    curr_bin = bin(curr_j)[2:].zfill(length)
    prev_bin = bin(prev_j)[2:].zfill(length)
    return [i for i, (x,y) in enumerate(zip(curr_bin,prev_bin)) if x!=y]

In [27]:
def amplitude_embedding(x):
    n = int(np.log2(len(x)))
    qubits = cirq.GridQubit.rect(1, n)
    circuit = cirq.Circuit()
    
    circuit += cirq.ry(beta(n, 1, x))(qubits[0])
    
    for i in range(1,n):
        # We can have at most i control bits
        # Total possibilities is therefore 2^i
        controls = 2**i
        
        control_qubits = [qubits[c] for c in range(i+1)]
        circuit += cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls, x)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)
        
        for j in range(1, controls):
            for loc in locate_x(controls-j-1, controls-j, i):
                circuit += cirq.X(qubits[loc])
                
                circuit += cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls-j, x)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)
            
        for k in range(i):
            circuit += cirq.X(qubits[k])

    return circuit

In [28]:
x = [1,2,3,4]
d = np.sqrt(np.sum(np.square(x)))
x = x/d
print('amplitudes:',x)
print('probs:',np.square(x))
qc = amplitude_embedding(x)

amplitudes: [0.18257419 0.36514837 0.54772256 0.73029674]
probs: [0.03333333 0.13333333 0.3        0.53333333]


In [29]:
qubits = cirq.GridQubit.rect(1, 2)
qc += cirq.measure(*qubits, key='result')
qc

(0, 0): ───Ry(0.732π)───@───────────X───@────────────X───M('result')───
                        │               │                │
(0, 1): ────────────────Ry(0.59π)───────Ry(0.705π)───────M─────────────

In [30]:
s=cirq.Simulator()
shots = 10000
samples=s.run(qc, repetitions=shots)
res = dict(samples.histogram(key="result"))
for key, value in res.items():
    res[key] = value/shots
res

{3: 0.5351, 1: 0.134, 2: 0.2976, 0: 0.0333}

## Using symbols

In [64]:
import sympy
from functools import reduce

In [152]:
def beta(s, j, x):
    index_num = (2*j-1)*(2**(s-1))
    index_den = (j-1)*(2**s)
    
    num_start = index_num
    num_end = index_num + 2**(s-1)
    
    den_start = index_den
    den_end = index_den + 2**(s)
    
    if((den_end <= den_start) or (num_end <= num_start)):
        return 0
    
    res = [sympy.Abs(x[i])**2 for i in range(num_start,num_end,1)]
    coeff = reduce(lambda m, n: m + n, res) 
    num_coeff = sympy.sqrt(coeff)
    
    res = [sympy.Abs(x[i])**2 for i in range(den_start,den_end,1)]
    coeff = reduce(lambda m, n: m + n, res) 
    den_coeff = sympy.sqrt(coeff)
    
    beta = 2*sympy.asin(num_coeff/den_coeff)
    
    return beta

In [153]:
def locate_x(curr_j, prev_j, length):
    curr_bin = bin(curr_j)[2:].zfill(length)
    prev_bin = bin(prev_j)[2:].zfill(length)
    return [i for i, (x,y) in enumerate(zip(curr_bin,prev_bin)) if x!=y]

In [154]:
def amplitude_embedding(qubits, symbols):
    n = len(qubits)
    ae_ops = []
    ae_ops += [cirq.ry(beta(n, 1, symbols))(qubits[0])]
    
    for i in range(1,n):
        # We can have at most i control bits
        # Total possibilities is therefore 2^i
        controls = 2**i
        
        control_qubits = [qubits[c] for c in range(i+1)]
        ae_ops += [cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls, symbols)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)]
        
        for j in range(1, controls):
            for loc in locate_x(controls-j-1, controls-j, i):
                ae_ops += [cirq.X(qubits[loc])]
                
                ae_ops += [cirq.ControlledGate(sub_gate=cirq.ry(beta(n-i, controls-j, symbols)), 
                                       num_controls=len(control_qubits)-1)(*control_qubits)]
            
        for k in range(i):
            ae_ops += [cirq.X(qubits[k])]

    return ae_ops

In [176]:
x = sympy.symbols(f'x0:{16}')
x = np.asarray(x).reshape((16))

In [177]:
qubits = cirq.GridQubit.rect(1,4)

In [178]:
ops = amplitude_embedding(qubits,x)

In [179]:
qc = cirq.Circuit()

In [180]:
qc += ops

In [181]:
qc

(0, 0): ───Ry(2*asin(sqrt(Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x8)**2 + Abs(x9)**2)/sqrt(Abs(x0)**2 + Abs(x1)**2 + Abs(x10)**2 + Abs(x11)**2 + Abs(x12)**2 + Abs(x13)**2 + Abs(x14)**2 + Abs(x15)**2 + Abs(x2)**2 + Abs(x3)**2 + Abs(x4)**2 + Abs(x5)**2 + Abs(x6)**2 + Abs(x7)**2 + Abs(x8)**2 + Abs(x9)**2)))───@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────X───@───────────────────────────────────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────X───@──────────────────────────────────────────────────────────@──────────────────────────────────────────────────────────@──────────────────────────────────────────────────────────@──────────────────────────────────────────────────────────@───────────────────────────────────────────────────X───@───────────────────────────────────────────────────────@───────────────────────────────────────────────────────@───────────────────────────────────────────────────────@───────────────────────────────────────────────────────@───────────────────────────────────────────────────────@───────────────────────────────────────────────────────@───────────────────────────────────────────────────X───
                                                                                                                                                                                                                                                                                                                                                                     │                                                                                                                                                                                               │                                                                                                                                                                                     │                                                                                                             │                                                                                                           │                                                                                                       │                                                                                                       │                                                                                                       │                                                          │                                                          │                                                          │                                                          │                                                       │                                                       │                                                       │                                                       │                                                       │                                                       │                                                       │
(0, 1): ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [210]:
y = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
y = np.array([
    y,y,y,y
])

In [211]:
y = normalize(y)

In [218]:
y

array([[0.02585438, 0.05170877, 0.07756315, 0.10341754, 0.12927192,
        0.15512631, 0.18098069, 0.20683508, 0.23268946, 0.25854384,
        0.28439823, 0.31025261, 0.336107  , 0.36196138, 0.38781577,
        0.41367015],
       [0.02585438, 0.05170877, 0.07756315, 0.10341754, 0.12927192,
        0.15512631, 0.18098069, 0.20683508, 0.23268946, 0.25854384,
        0.28439823, 0.31025261, 0.336107  , 0.36196138, 0.38781577,
        0.41367015],
       [0.02585438, 0.05170877, 0.07756315, 0.10341754, 0.12927192,
        0.15512631, 0.18098069, 0.20683508, 0.23268946, 0.25854384,
        0.28439823, 0.31025261, 0.336107  , 0.36196138, 0.38781577,
        0.41367015],
       [0.02585438, 0.05170877, 0.07756315, 0.10341754, 0.12927192,
        0.15512631, 0.18098069, 0.20683508, 0.23268946, 0.25854384,
        0.28439823, 0.31025261, 0.336107  , 0.36196138, 0.38781577,
        0.41367015]])

In [189]:
resolver = cirq.ParamResolver({i:y[0][index]for index, i in enumerate(x)})
output_state_vector = cirq.Simulator().simulate(qc, resolver).final_state_vector
output_state_vector

array([ 0.06536265+0.j,  0.01838325+0.j,  0.0453367 +0.j,  0.06800506+0.j,
       -0.03388227+0.j,  0.17865191+0.j,  0.17038853+0.j,  0.19878663+0.j,
       -0.03629497+0.j,  0.34002668+0.j,  0.28398094+0.j,  0.312379  +0.j,
        0.3407771 +0.j,  0.36917517+0.j,  0.39757332+0.j,  0.4259714 +0.j],
      dtype=complex64)

In [206]:
import pandas as pd

In [212]:
df = pd.DataFrame(y)
resolver = cirq.ListSweep(np.array([i for i in df.T.to_dict().values()]))

In [213]:
resolver

cirq.ListSweep([cirq.ParamResolver({0: 0.025854384499750957, 1: 0.051708768999501914, 2: 0.07756315349925287, 3: 0.10341753799900383, 4: 0.1292719224987548, 5: 0.15512630699850574, 6: 0.1809806914982567, 7: 0.20683507599800766, 8: 0.23268946049775863, 9: 0.2585438449975096, 10: 0.28439822949726057, 11: 0.3102526139970115, 12: 0.33610699849676245, 13: 0.3619613829965134, 14: 0.3878157674962644, 15: 0.4136701519960153}), cirq.ParamResolver({0: 0.025854384499750957, 1: 0.051708768999501914, 2: 0.07756315349925287, 3: 0.10341753799900383, 4: 0.1292719224987548, 5: 0.15512630699850574, 6: 0.1809806914982567, 7: 0.20683507599800766, 8: 0.23268946049775863, 9: 0.2585438449975096, 10: 0.28439822949726057, 11: 0.3102526139970115, 12: 0.33610699849676245, 13: 0.3619613829965134, 14: 0.3878157674962644, 15: 0.4136701519960153}), cirq.ParamResolver({0: 0.025854384499750957, 1: 0.051708768999501914, 2: 0.07756315349925287, 3: 0.10341753799900383, 4: 0.1292719224987548, 5: 0.15512630699850574, 6: 0.

In [217]:
cirq.protocols.resolve_parameters(y, param_resolver=resolver)

array([[0.02585438, 0.05170877, 0.07756315, 0.10341754, 0.12927192,
        0.15512631, 0.18098069, 0.20683508, 0.23268946, 0.25854384,
        0.28439823, 0.31025261, 0.336107  , 0.36196138, 0.38781577,
        0.41367015],
       [0.02585438, 0.05170877, 0.07756315, 0.10341754, 0.12927192,
        0.15512631, 0.18098069, 0.20683508, 0.23268946, 0.25854384,
        0.28439823, 0.31025261, 0.336107  , 0.36196138, 0.38781577,
        0.41367015],
       [0.02585438, 0.05170877, 0.07756315, 0.10341754, 0.12927192,
        0.15512631, 0.18098069, 0.20683508, 0.23268946, 0.25854384,
        0.28439823, 0.31025261, 0.336107  , 0.36196138, 0.38781577,
        0.41367015],
       [0.02585438, 0.05170877, 0.07756315, 0.10341754, 0.12927192,
        0.15512631, 0.18098069, 0.20683508, 0.23268946, 0.25854384,
        0.28439823, 0.31025261, 0.336107  , 0.36196138, 0.38781577,
        0.41367015]])